### 蒸馏模型对比测试流程

In [19]:
prompt = "请问X平方+1=2，X等于几？"

In [51]:
prompt = "设函数 f(x)= x^3 - 6x^2 + 9x + 2，求;\
1.f(x)的极值点;\
2.在这些极值点处的函数值;\
3.f(x)在这些极值点处的单调性。" 

In [10]:
prompt = """
假设有 7 个海盗，他们需要分配一批金币。每个海盗都具有以下特点：

1. **绝对理性**：每个海盗都会尽量使自己的利益最大化。
2. **绝对自私**：他们只关心自己的得失，不关心他人的情况。
3. **暴力威胁**：如果一个提案无法被通过（投票否决），则提议者会被“扔下船”。

**分赃规则**

1. 按照从首领（编号为 1 的海盗）到最后一名海盗的顺序，依次由每个海盗提出分赃方案。
2. 每名海盗（包括提议者）对提案进行投票。提案若获得 **至少一半人数（包括自己）或以上** 的支持，则通过；否则，提议者被抛弃，剩下的海盗继续讨论。
3. 海盗之间按照逻辑进行投票，遵循理性自私的行为模式。

**目标**

首领（提议者）需要提出一个分赃方案，使：

1. 自己不会被扔下船（提案被通过）。
2. 自己分得的金币尽可能多。

问，他们应该如何分赃
"""

In [25]:
prompt = """
写一个 Bash 脚本，将格式为 '[1,2],[3,4],[5,6]' 的字符串表示的矩阵转置，并以相同格式输出
"""

In [42]:
prompt = """
证明n趋于无穷时，1/(n^2)为0.
"""

In [2]:
# 测试问题
prompt = """
实现一个使用 HTML + JS + CSS 的贪吃蛇游戏。
整个代码应该写在一个单一的 HTML 块中，嵌入 JS 和 CSS。不要使用任何远程资源。
打开 HTML 后，用户需要按空格键开始/重新开始游戏，蛇在开始时会随机朝一个方向移动，并使用 "wasd" 键来控制蛇的方向。
让游戏界面美观，并使游戏区域较大。
"""

---

- 原始模型回答流程

In [11]:
# 普通模型
path1 = "/root/autodl-tmp/DeepSeek-R1-Distill/Qwen2.5-1.5B-Instruct"

In [12]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

In [13]:
# 输入模型下载地址
model_name = path1

In [14]:
# 实例化预训练模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
# 创建消息
# prompt = "你好，好久不见，请介绍下你自己。"
messages = [
    {"role": "system", "content": "你是一名助人为乐的助手。"},
    {"role": "user", "content": prompt}
]

In [16]:
# 分词
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [17]:
# 创建回复
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=16000
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [18]:
print(response)

在海盗分赃问题中，为了确保首领（提议者）能够得到最多金币且自身不会被丢弃，可以采取以下策略：

### 分赃计划

1. **前两名海盗（首领和第二名海盗）**：
   - 第一海盗（首领）将拥有所有金币，并将其平均分给第二名海盗。
   - 第二名海盗（第二名海盗）会同意这一分配，因为这样他能拿到第一海盗所有的金币，而首领是绝对理性的，他会选择这种分配以确保自己不会被丢弃。

2. **剩余的五名海盗（第三到第七名海盗）**：
   - 剩余的五名海盗将会采用相同的逻辑，即第二名海盗会接受第一名海盗的分配方案，因为他能得到最多的金币。
   - 这样，第三名海盗会接受第二名海盗的分配方案，以此类推，直到第七名海盗。

因此，最终的分配方案如下：

- 首领（第1名海盗）：所有金币
- 第二名海盗（第2名海盗）：所有金币的一半

这样，首领既能保证自己不会被丢弃，又能得到最多的金币。


- 蒸馏模型回答流程

In [19]:
# 蒸馏模型
path2 = "/root/autodl-tmp/DeepSeek-R1-Distill/Qwen2.5-1.5B-Distill"

In [20]:
# 输入模型下载地址
model_name = path2

In [21]:
# 实例化预训练模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [22]:
# 创建消息
# prompt = "你好，好久不见，请介绍下你自己。"
messages = [
    {"role": "system", "content": "你是一名助人为乐的助手。"},
    {"role": "user", "content": prompt}
]

In [23]:
messages

[{'role': 'system', 'content': '你是一名助人为乐的助手。'},
 {'role': 'user',
  'content': '\n假设有 7 个海盗，他们需要分配一批金币。每个海盗都具有以下特点：\n\n1. **绝对理性**：每个海盗都会尽量使自己的利益最大化。\n2. **绝对自私**：他们只关心自己的得失，不关心他人的情况。\n3. **暴力威胁**：如果一个提案无法被通过（投票否决），则提议者会被“扔下船”。\n\n**分赃规则**\n\n1. 按照从首领（编号为 1 的海盗）到最后一名海盗的顺序，依次由每个海盗提出分赃方案。\n2. 每名海盗（包括提议者）对提案进行投票。提案若获得 **至少一半人数（包括自己）或以上** 的支持，则通过；否则，提议者被抛弃，剩下的海盗继续讨论。\n3. 海盗之间按照逻辑进行投票，遵循理性自私的行为模式。\n\n**目标**\n\n首领（提议者）需要提出一个分赃方案，使：\n\n1. 自己不会被扔下船（提案被通过）。\n2. 自己分得的金币尽可能多。\n\n问，他们应该如何分赃\n'}]

In [24]:
# 分词
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [25]:
# 创建回复
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=16000
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [26]:
print(response)

在这样的海盗分赃问题中，我们需要确保首领能够提出一个提案，并且这个提案能够得到至少一半的支持。同时，首领希望自己的份额尽可能多。由于是绝对理性和绝对自私的海盗，他们会根据自己的利益来决定是否接受提案。

首先，让我们明确一下所有海盗的特征和规则：

- 理性、自私的海盗会考虑自己的利益最大化。
- 提案如果不能得到超过半数的支持，那么提议者就会被抛弃，剩下的海盗会重新开始讨论。

目标：

- 第1号海盗要确保自己的提案能够得到至少半数的支持。
- 同时，第1号海盗希望自己分得的金币尽可能多。

### 分析步骤

1. **确定可能的选择**：
   - 可能的分赃方案有多种，比如按比例分配、固定数量等。
   - 首先，考虑最简单的方案，即固定金币的数量给每个海盗。

2. **考虑不同情况**：
   - 如果金币数量少于海盗数量的一半，那么只有首领能得到全部金币。
   - 如果金币数量等于海盗数量的一半，那么每个人都能得到相同数量的金币。
   - 如果金币数量多于海盗数量的一半，那么可以分成两部分：一部分给首领，另一部分给剩余的海盗。

3. **考虑首领的利益**：
   - 首领的目标是最大化自己的利益，即最大化自己的金币份额，同时确保自己的提案能够通过。

### 推测方案

基于上述分析，我推测一种可能的分赃方案：

- **首领（1号）**：得到最多金币，比如50%或者更多。
- **第二号到第六号**：得到较少的金币，但为了生存，他们可能会接受较低的份额以避免被抛弃。
- **第七号**：得到最少的金币，或者没有任何金币，因为他是最后一名。

具体来说：

1. **首领**：得到大部分金币，比如60%。
2. **第二号**：得到20%。
3. **第三号到第五号**：各得到10%。
4. **第六号**：再分一些给第三号到第五号，因为他们是首领之前的人，可能愿意接受较低的份额。
5. **第七号**：什么也不得到，因为他的提案没有通过。

### 议程

1. **第一轮**：首领提出一个提案，例如60%给首领，其他50%分给第二号到第七号。
2. **第二轮**：第二号到第七号对提案进行投票。假设他们中有四人同意，三人反对。首领会失去这次投票的权利，剩下的海盗继续讨论。
3. **第三轮**：首领再次尝试，但已经失去了投票权，所以只能等